In [1]:
from mplsoccer import Pitch, FontManager, Sbopen, VerticalPitch
import numpy as np
from matplotlib.colors import to_rgba
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
# Spain vs Poland
MATCH_ID = 3788762
TEAM = 'Poland'
parser = Sbopen()
events, related, freeze, players = parser.event(MATCH_ID)

In [6]:
events.columns

Index(['id', 'index', 'period', 'timestamp', 'minute', 'second', 'possession',
       'duration', 'match_id', 'type_id', 'type_name', 'possession_team_id',
       'possession_team_name', 'play_pattern_id', 'play_pattern_name',
       'team_id', 'team_name', 'tactics_formation', 'player_id', 'player_name',
       'position_id', 'position_name', 'pass_recipient_id',
       'pass_recipient_name', 'pass_length', 'pass_angle', 'pass_height_id',
       'pass_height_name', 'end_x', 'end_y', 'sub_type_id', 'sub_type_name',
       'body_part_id', 'body_part_name', 'x', 'y', 'outcome_id',
       'outcome_name', 'out', 'under_pressure',
       'ball_recovery_recovery_failure', 'pass_deflected', 'aerial_won',
       'counterpress', 'pass_switch', 'off_camera', 'foul_won_defensive',
       'shot_statsbomb_xg', 'end_z', 'technique_id', 'technique_name',
       'goalkeeper_position_id', 'goalkeeper_position_name', 'pass_cross',
       'pass_cut_back', 'pass_assisted_shot_id', 'pass_shot_assist',
    

In [7]:
events.groupby('type_name').id.unique()

type_name
Ball Receipt         [131decb1-6673-4d38-b9d8-17744d4d7c92, bd96c81...
Ball Recovery        [d399b421-01a3-4808-b01d-d6a2a070f03a, ebedd46...
Block                [8a9d7d80-7075-4af7-9444-ded43f5cba41, 160fba0...
Carry                [f93912a1-9dd9-41f3-a7d2-a3f1ffd4584e, 1420e27...
Clearance            [271ea3e2-8e41-45d4-9de5-d722fc40a413, 1a801d3...
Dispossessed         [57c6e562-a748-4b56-9a28-c4d68a35d8de, 1694703...
Dribble              [4ea55b4d-4488-4645-86ca-1677ef1324a6, 02acc17...
Dribbled Past        [dc026c40-111e-4289-95b7-c7228f2e4d6b, 4e6dbcf...
Duel                 [a3f7d238-f87d-417b-a844-05a334151621, 273c594...
Error                           [d534c33f-d7b3-4972-b16a-b921d68456c5]
Foul Committed       [2c6fd056-5095-4507-86b7-7c13d9c90acb, 358ba49...
Foul Won             [228c5124-53ec-40f0-b010-7373be42754f, 6c9e12d...
Goal Keeper          [0eddac9e-a1b3-46a4-b506-a3e9a6eff725, af65f17...
Half End             [50b33666-949c-49ed-9836-ddd7cd3aeffc, 39c00cd

In [8]:
events.loc[events.tactics_formation.notnull(), 'tactics_id'] = events.loc[events.tactics_formation.notnull(), 'id']
events[['tactics_id', 'tactics_formation']] = events.groupby('team_name')[['tactics_id', 'tactics_formation']].ffill()

In [10]:
formation_dict = {1: 'GK', 2: 'RB', 3: 'RCB', 4: 'CB', 5: 'LCB', 6: 'LB', 7: 'RWB',
                  8: 'LWB', 9: 'RDM', 10: 'CDM', 11: 'LDM', 12: 'RM', 13: 'RCM',
                  14: 'CM', 15: 'LCM', 16: 'LM', 17: 'RW', 18: 'RAM', 19: 'CAM',
                  20: 'LAM', 21: 'LW', 22: 'RCF', 23: 'ST', 24: 'LCF', 25: 'SS'}
players['position_abbreviation'] = players.position_id.map(formation_dict)

In [11]:
sub = events.loc[events.type_name == 'Substitution',
                 ['tactics_id', 'player_id', 'substitution_replacement_id',
                  'substitution_replacement_name']]
players_sub = players.merge(sub.rename({'tactics_id': 'id'}, axis='columns'),
                            on=['id', 'player_id'], how='inner', validate='1:1')
players_sub = (players_sub[['id', 'substitution_replacement_id', 'position_abbreviation']]
               .rename({'substitution_replacement_id': 'player_id'}, axis='columns'))
players = pd.concat([players, players_sub])
players.rename({'id': 'tactics_id'}, axis='columns', inplace=True)
players = players[['tactics_id', 'player_id', 'position_abbreviation']]

In [12]:
# add on the position the player was playing in the formation to the events dataframe
events = events.merge(players, on=['tactics_id', 'player_id'], how='left', validate='m:1')
# add on the position the receipient was playing in the formation to the events dataframe
events = events.merge(players.rename({'player_id': 'pass_recipient_id'},
                                    axis='columns'), 
                                    on=['tactics_id', 'pass_recipient_id'],
                                    how='left', 
                                    validate='m:1', 
                                    suffixes=['', '_receipt'])

events['tactics_formation'] = events['tactics_formation'].astype(int)

In [13]:
events.groupby('team_name').tactics_formation.unique()

team_name
Poland    [352, 343]
Spain          [433]
Name: tactics_formation, dtype: object

In [19]:
FORMATION = 352
pass_cols = ['id', 'position_abbreviation', 'position_abbreviation_receipt']
passes_formation = events.loc[(events.team_name == TEAM) & (events.type_name == 'Pass') & #(events.type_name == 'Dribble') & (events.type_name == 'Shot') &
                              (events.tactics_formation == FORMATION) &
                              (events.position_abbreviation_receipt.notnull()), pass_cols].copy()
location_cols = ['position_abbreviation', 'x', 'y']
location_formation = events.loc[(events.team_name == TEAM) &
                                (events.type_name.isin(['Pass', 'Ball Receipt'])) &
                                (events.tactics_formation == FORMATION), location_cols].copy()




In [21]:
events.columns

Index(['id', 'index', 'period', 'timestamp', 'minute', 'second', 'possession',
       'duration', 'match_id', 'type_id', 'type_name', 'possession_team_id',
       'possession_team_name', 'play_pattern_id', 'play_pattern_name',
       'team_id', 'team_name', 'tactics_formation', 'player_id', 'player_name',
       'position_id', 'position_name', 'pass_recipient_id',
       'pass_recipient_name', 'pass_length', 'pass_angle', 'pass_height_id',
       'pass_height_name', 'end_x', 'end_y', 'sub_type_id', 'sub_type_name',
       'body_part_id', 'body_part_name', 'x', 'y', 'outcome_id',
       'outcome_name', 'out', 'under_pressure',
       'ball_recovery_recovery_failure', 'pass_deflected', 'aerial_won',
       'counterpress', 'pass_switch', 'off_camera', 'foul_won_defensive',
       'shot_statsbomb_xg', 'end_z', 'technique_id', 'technique_name',
       'goalkeeper_position_id', 'goalkeeper_position_name', 'pass_cross',
       'pass_cut_back', 'pass_assisted_shot_id', 'pass_shot_assist',
    

In [20]:
passes_formation

,id,position_abbreviation,position_abbreviation_receipt
4,45b27e4b-eae8-4c74-8ac1-d8cd1b53e0bb,CDM,LCM
7,14eae7de-09ab-44c8-abfa-dee73a95f0fc,LCM,RCF
11,09e0d59c-6eb3-4440-84ac-40c6114a5f59,RWB,CDM
14,d6c2ce83-69ea-4e79-8bac-ebfe225c1a82,CDM,RWB
17,e31ab7dd-d4a9-4116-ae0a-4cfe1d48050e,RWB,CDM
...,...,...,...
2470,faebf7d2-1ddd-49ce-936e-f99e091e9011,LCB,GK
2497,74de4a94-634f-442a-8898-a206ac3f05f1,RWB,CDM
2501,aef9ea42-402b-4446-a527-bdb38b900976,CDM,RWB
2512,6d4b2548-f365-4fd6-bd10-841e88a2d194,LWB,CDM
